import hgvs
from hgvs.easy import *
https://stackoverflow.com/questions/62807717/how-can-i-solve-postgresql-scram-authentication-problem

In [ ]:
# resources 
# https://github.com/ga4gh/vrs-python/blob/a40fa88f75983fc65ec8da92d044bff67b307175/notebooks/HGVS%20Translation.ipynb
# https://github.com/ga4gh/vrs-python/blob/main/notebooks/Extras.ipynb
# https://github.com/ga4gh/vrs-python/blob/a23bfcb41176e702523e5f457cf28cc8087f2fd4/src/ga4gh/vrs/extras/variation_normalizer_rest_dp.py

In [ ]:
from ga4gh.vrs import __version__, models, normalize
from ga4gh.core import ga4gh_digest, ga4gh_identify, ga4gh_serialize
from ga4gh.vrs.dataproxy import SeqRepoRESTDataProxy
from biocommons.seqrepo import SeqRepo
import json
from ga4gh.vrs.extras.variation_normalizer_rest_dp import VariationNormalizerRESTDataProxy
from variation_services_rest_db import VariationServicesRESTDataProxy
vnorm = VariationNormalizerRESTDataProxy()
vs = VariationServicesRESTDataProxy()

: 

In [ ]:
def pjson(o, indent=2):
    """pretty print object as json"""
    return json.dumps(o.as_dict(), sort_keys=True, indent=indent)

In [ ]:
def ppo(o, indent=2):
    """pretty print object as json"""
    return json.dumps(o.as_dict(), sort_keys=True, indent=indent)

In [5]:
seqrepo_rest_service_url = "https://services.genomicmedlab.org/seqrepo"
dp = SeqRepoRESTDataProxy(base_url=seqrepo_rest_service_url)

In [3]:
from ga4gh.vrs.extras.translator import Translator
tlr = Translator(data_proxy=dp,
                 translate_sequence_identifiers=True,  # default
                 normalize=True,                       # default
                 identify=True)                        # default

In [ ]:
# input SPDI nomenclature 
#spdi_ex = 'NC_000006.12:31574704:C:T' # other example "NC_000006.12:31574704:C:T", NC_000019.10:44908821:1:T
# spdi_exprs =[
#     "NC_000017.11:7674796:T:A", # TP53
#     "NC_000006.12:31574704:C:T", # TNF 
#     "NC_000019.10:44908821:C:T", # APOE
#     "NC_000019.10:41354681:C:T" # TGFB1 
# ] 

In [ ]:
# test = 'NG_012345.1,4,1,'
# refseq,pos,ref,alt = test.split(',')
# spdi_test_example = '{}:{}:{}:{}'.format(refseq,pos,ref,alt)
# print(spdi_test_example)

In [ ]:
#allele = tlr.translate_from(spdi_test_example)
# # output VRS-Allele-Object
# allele.as_dict()

In [ ]:
# not this method is replacing translate_to
# HGVS Normalized
# vnorm.to_hgvs(allele, 'refseq')

In [4]:
import csv
# Load the test_spdi table
with open('/infodev1/cim/Salem/ga4ghprojects/vrs-python/notebooks/data/spdi_test_data.csv', 'r') as f:
    reader = csv.reader(f)
    spdi_id_list = []
    for row in reader:
        records = spdi_id_list.append(':'.join(row))

header = spdi_id_list.pop(0)


In [ ]:
# def ppo(o):
#     """pretty print object as json"""
#     print(json.dumps(o.as_dict(), indent=2))

# alleles = []
# vrs_id = []
# vrs_alleles = []
# for expr in spdi_id_list:
#     try:
#         trans = tlr.translate_from(expr)
#         alleles.append(trans)
#         vrs_id.append(ga4gh_identify(trans))
#         vrs_alleles.append(pjson(trans))
#         #print(vrs_id, pjson(trans), sep = '\n')
#     except:
#         #print('{0} {1}'.format('ERROR in the syntext',expr))
#         alleles.append('{0} {1}'.format('ERROR in the syntext',expr))

In [5]:
spdi_id_list

['NG_012345.1:4:1:T',
 'NG_012345.1:4:1:',
 'NG_012345.1:4:0:T',
 'NG_012345::0:T',
 'NC_000010.11:87925523:C:G']

In [9]:
rshifthgvs = []

for spdi_expr in spdi_id_list:
    try: 
        # Converting a allele in SPDI syntax to the right-shifted HGVS notation
        rshifthgvs.append(vs.spdi2hgvs(spdi_expr))
    except: 
        rshifthgvs.append('{0} {1}'.format('ERROR in the syntext',spdi_expr))

print(rshifthgvs)

['NG_012345.1:g.5G>T', 'NG_012345.1:g.5del', 'NG_012345.1:g.4_5insT', 'ERROR in the syntext NG_012345::0:T', 'NC_000010.11:g.87925524C>G']


In [ ]:
alleles = []

for expr in spdi_id_list:
    try:
        trans = tlr.translate_from(expr)
        #vrs_allele_dict[ga4gh_identify(trans)] = pjson(trans)
        alleles.append(trans)
        #print(vrs_id, pjson(trans), sep = '\n')
    except:
        #print('{0} {1}'.format('ERROR in the syntext',expr))
        alleles.append('{0} {1}'.format('ERROR in the syntext',expr))
        #vrs_allele_dict["ERROR in the syntext"] = expr
print(alleles)

In [ ]:
vrs_allele_dict = {}

for expr in spdi_id_list:
    try:
        trans = tlr.translate_from(expr)
        # TODO: make sure this json.dumps is actually working
        vrs_allele_dict[ga4gh_identify(trans)] = json.dumps(trans.as_dict())
    except:
        vrs_allele_dict["ERROR in the syntext"] = expr

print(vrs_allele_dict.keys())

In [ ]:
norm_hgvs_exprs = []
for allele in alleles:
    if isinstance(allele, str):
        norm_hgvs_exprs.append('[{0} {1}]'.format('ERROR in the syntext',expr))
    else: 
        vrs_allele = vnorm.to_hgvs(allele, 'refseq')
        norm_hgvs_exprs.append(vrs_allele)
        
print(norm_hgvs_exprs)

In [ ]:
# spdi_id_list,vrs_allele_dict.keys(),vrs_allele_dict,rshifthgvs,norm_hgvs_exprs

In [ ]:
# print(norm_hgvs_exprs)

In [ ]:
print( len(alleles),len(vrs_allele_dict.keys()), len(norm_hgvs_exprs) )

In [ ]:
import pandas as pd

mydict = {'spdi_expression': spdi_id_list,
          #'ga4gh_identifier': vrs_allele_dict.keys(),
        'rightshift_hgvs_expression': rshifthgvs, 
        'normalized_hgvs_expressions': norm_hgvs_exprs}


df1 = pd.DataFrame(mydict)
 
df1

In [ ]:
df2 = pd.DataFrame(vrs_allele_dict.items(), columns=['ga4gh_id','vrs_allele_obejct'])
df2

In [ ]:
frames = [df1, df2]

result = pd.concat(frames,axis = 1)

In [ ]:
look in to this

In [ ]:
from IPython.display import HTML, display
import tabulate

In [ ]:
# Translator with default behaviors disabled
tlr2 = Translator(data_proxy=dp,
                  translate_sequence_identifiers=True,
                  normalize=False,
                  identify=False)

In [ ]:
hgvs_expr_test = 'NG_012345.1:g.5G>T'
header = "normalize= sequence_id interval alt hgvs".split()
table = [header]
for normalize in (False, True):
    tlr2.normalize = normalize
    a = tlr2.translate_from(hgvs_expr_test, "hgvs")
    row = [tlr2.normalize,
           a.location.sequence_id,
           f"{a.location.interval.start.value},{a.location.interval.end.value}",
           a.state.sequence,
           #tlr2.translate_to(a, 'hgvs')[0]
           vnorm.to_hgvs(a)[0]
          ]
    table += [row]
display(HTML(tabulate.tabulate(table, tablefmt='html')))

In [ ]:
# result.to_csv('/infodev1/cim/Salem/ga4ghprojects/vrs-python/notebooks/data/output_test.csv') 

In [ ]:
#print(json.dumps(df2.vrs_allele_obejct[0]))

In [ ]:
# norm_hgvs_exprs = []
# for allele in alleles:
#     if type(allele) == "abc.Allele":
#         vrs_allele = vnorm.to_hgvs(allele, 'refseq')
#         norm_hgvs_exprs.append(vrs_allele)
#         print(vrs_allele)
#     else: 
#          norm_hgvs_exprs.append(allele)